In [1]:
import os
import time
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter


In [2]:
config = {}

# Fname 
config.update({"FolderName": "2023_0303"})

# Fpath
config.update({"FolderPath": "/Volumes/Expansion/User_Backup/b08209033/111-2_IVT_analysis"})
config.update({"SubFolderPath": os.path.join(config["FolderPath"], config["FolderName"])})
config.update({"DataPath": "/Volumes/Expansion/DATA/Reanalysis/ERA5"})
config.update({"SrcPath": os.path.join(config["SubFolderPath"], "src")})
config.update({"ImgPath": os.path.join(config["SubFolderPath"], "img")})

# Data slicing
config.update({"crop_region": [[0, 40], [60, 120]]})
config.update({"crop_year": [1979, 2021]})
config.update({"crop_month": [5, 7]})


os.chdir(config["FolderPath"])

In [3]:
for key, value in config.items():
    print(f"{key}: {value}")

FolderName: 2023_0303
UserPath: /Volumes/Expansion/User_Backup/b08209033
FolderPath: /Volumes/Expansion/User_Backup/b08209033/2023_0303
DataPath: /Volumes/Expansion/DATA/Reanalysis/ERA5
SrcPath: /Volumes/Expansion/User_Backup/b08209033/2023_0303/src
ImgPath: /Volumes/Expansion/User_Backup/b08209033/2023_0303/img
crop_region: [[0, 40], [60, 120]]
crop_year: [1979, 2021]
crop_month: [5, 7]


In [4]:
def createFolder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        print("Folder already existed.")
createFolder(config["SubFolderPath"])
createFolder(config["SrcPath"])
createFolder(config["ImgPath"])

Folder already existed.
Folder already existed.


In [5]:
def MonthDayIndex(year, month):
    idx = 0
    if (year%4==0):
        MD = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
    else:
        MD = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    for i in range(month-1):
        idx += MD[i]
    return idx

In [6]:
# return with dimensions (year, day, level, lat, lon)
def IVT(year, current_month, region_box):
    
    # Determine array shape
    os.chdir(os.path.join(config["DataPath"], 'u'))
    rootgrp = nc.Dataset('u1000_2010.nc')
        # Find region index
    lon = rootgrp['lon'][:]
    lon_idx = np.arange(np.argmax(lon >= region_box[1][0]), np.argmax(lon >= region_box[1][1])+1)
    lon = lon[lon_idx]
    lat = rootgrp['lat'][:]
    lat_idx = np.arange(np.argmax(lat >= region_box[0][0]), np.argmax(lat >= region_box[0][1])+1)
    lat = lat[lat_idx]
    rootgrp.close()
        # Prescribe level index
    level = np.array([1000, 925, 850, 700, 500, 250])
        # Find time index
        # Time may vary with leap year, so here is for array shape
    time = np.arange(MonthDayIndex(year[0], current_month[0]), MonthDayIndex(year[0], current_month[1]))
    
    # Basic variables
    u = np.array([]).reshape(0, len(time), len(level), len(lat), len(lon))
    v = np.array([]).reshape(0, len(time), len(level), len(lat), len(lon))
    q = np.array([]).reshape(0, len(time), len(level), len(lat), len(lon))
    
    # Read u wind
        # Inner loop: Concatenate with new dimensions (level)
        # Outer loop: Concatenate with new dimensions (year)
    os.chdir(os.path.join(config["DataPath"], 'u'))
    for yr in year:
        temp = np.array([]).reshape(len(time), 0, len(lat), len(lon))
        time_idx = np.arange(MonthDayIndex(yr, current_month[0]), MonthDayIndex(yr, current_month[1]))
        for lv in level:
            # Read file and variables
            fname = 'u'+ str(lv) + '_' + str(yr) + '.nc'
            rootgrp = nc.Dataset(fname)
            var = rootgrp['u'][time_idx,lat_idx,lon_idx]
            var = var.reshape(len(time), 1, len(lat), len(lon))
            # Concatenate
            temp = np.concatenate((temp, var), axis=1)
            rootgrp.close()
        temp = temp.reshape(1, len(time), len(level), len(lat), len(lon))
        u = np.concatenate((u, temp), axis=0)
    
    # Read v wind
        # Inner loop: Concatenate with new dimensions (level)
        # Outer loop: Concatenate with new dimensions (year)
    os.chdir(os.path.join(config["DataPath"], 'v'))
    for yr in year:
        temp = np.array([]).reshape(len(time), 0, len(lat), len(lon))
        time_idx = np.arange(MonthDayIndex(yr, current_month[0]), MonthDayIndex(yr, current_month[1]))
        for lv in level:
            # Read file and variables
            fname = 'v'+ str(lv) + '_' + str(yr) + '.nc'
            rootgrp = nc.Dataset(fname)
            var = rootgrp['v'][time_idx,lat_idx,lon_idx]
            var = var.reshape(len(time), 1, len(lat), len(lon))
            # Concatenate
            temp = np.concatenate((temp, var), axis=1)
            rootgrp.close()
        temp = temp.reshape(1, len(time), len(level), len(lat), len(lon))
        v = np.concatenate((v, temp), axis=0)
    
    # Read q wind
        # Inner loop: Concatenate with new dimensions (level)
        # Outer loop: Concatenate with new dimensions (year)
    os.chdir(os.path.join(config["DataPath"], 'q'))
    for yr in year:
        temp = np.array([]).reshape(len(time), 0, len(lat), len(lon))
        time_idx = np.arange(MonthDayIndex(yr, current_month[0]), MonthDayIndex(yr, current_month[1]))
        for lv in level:
            # Read file and variables
            fname = 'q'+ str(lv) + '_' + str(yr) + '.nc'
            rootgrp = nc.Dataset(fname)
            var = rootgrp['q'][time_idx,lat_idx,lon_idx]
            var = var.reshape(len(time), 1, len(lat), len(lon))
            # Concatenate
            temp = np.concatenate((temp, var), axis=1)
            rootgrp.close()
        temp = temp.reshape(1, len(time), len(level), len(lat), len(lon))
        q = np.concatenate((q, temp), axis=0)
    
    # Calculate IVT
        # Calculate interpolated value by averaging
    interp_u = (u[:,:,1:,:,:] + u[:,:,:-1,:,:])/2
    interp_v = (v[:,:,1:,:,:] + v[:,:,:-1,:,:])/2
    interp_q = (q[:,:,1:,:,:] + q[:,:,:-1,:,:])/2
        # hPa to Pa unit conversion
    thickness = (level[:-1] - level[1:])*100
        # Broadcast 
    thickness = thickness[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]
        # Integrate over levels
    IVT_u = np.sum((interp_u * interp_q * thickness)/9.81, axis = 2)
    IVT_v = np.sum((interp_v * interp_q * thickness)/9.81, axis = 2)
    
    return lat, lon, [IVT_u, IVT_v]

In [7]:
# Process
count = time.time()
lat, lon, IVT = IVT(np.arange(config['crop_year'][0], config['crop_year'][1]+1), config['crop_month'], config['crop_region'])
print(f"Takes {(time.time()-count):.4f} sec.")

Takes 226.6561 sec.


In [8]:

os.chdir(config['ImgPath'])
for yr in range(43):
    for day in range(61):
        fig, ax = plt.subplots(figsize = (10, 10),
                               dpi = 150,
                               subplot_kw = {'projection' : ccrs.PlateCarree()})
        ax.coastlines()
        ax.quiver(lon[::7], lat[::7], IVT[0][yr,day,::7,::7], IVT[1][yr,day,::7,::7])
        ax.set_xticks(np.linspace(config['crop_region'][1][0], config['crop_region'][1][1], 7), crs=ccrs.PlateCarree())
        ax.set_yticks(np.linspace(config['crop_region'][0][0], config['crop_region'][0][1], 5), crs=ccrs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True,number_format='.0f')
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        
        if day<31:
            plt.title(f'{yr+1979} May {day+1:02}', fontsize=16)
            #plt.savefig(f'{yr+1979}_05_{day+1:02}.png')
        else:
            plt.title(f'{yr+1979} June {day%31+1:02}', fontsize=16)
            #plt.savefig(f'{yr+1979}_06_{day%31+1:02}.png')
        plt.close()
